<a href="https://colab.research.google.com/github/somtirthadas/Python_project_3rd_sem/blob/main/test_llama_ft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL REQUIRED MODELS

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


IMPORT MODULES

In [2]:
import os
import torch
#from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    #HfArgumentParser,
    #TrainingArguments,
    pipeline,
    logging,
)
#from peft import LoraConfig, PeftModel
#from trl import SFTTrainer

GPU CONFIGURATION

In [3]:
import accelerate
device_map = {"": 0}

FIX MODEL CONFIGURATIONS

In [4]:
model_name = "somtirthadas2004/Llama-2-7b-chat-finetune2"

In [5]:
use_4bit = True                         #Activate 4 bit precision
bnb_4bit_compute_dtype = "float16"      #Compute data type
bnb_4bit_quant_type = "nf4"             #quantization type (fp4 or nf4)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config, device_map = device_map)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [9]:
stop_words=("exit", "good Bye", "not required", "no", "no thanks", "thank you", "Thank you", "EXIT", "Exit", "No Thanks", "Not Needed", "No", "Bye")

In [10]:
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, temperature=0.6, top_p=0.9, max_length=300)

flag1=0
flag2=0

print("Hi welcome to Career Mitra, your own career help chat-bot.")
while True:
  print("To start with us, tell me your current working field or area of interest.")
  prompt1=input()
  print("Good, now in that context what are the skills that you have?")
  prompt2=input()
  result = pipe(f"<s>[INST] With this area of interest {prompt1} and skillset {prompt2} what can be my suitable career? [/INST]")
  print(result[0]['generated_text'])
  while True:
    print("Any further queries or clarification you need regarding my response?")
    prompt=input()
    for words in stop_words:
      if prompt == words:
        print("Thank you, Have a nice day.")
        flag1 = 1
        flag2 = 1
        break
    if flag2 == 1:
      break
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])
  if flag1 == 1:
    break

Hi welcome to Career Mitra, your own career help chat-bot.
To start with us, tell me your current working field or area of interest.
Btech Computer Science
Good, now in that context what are the skills that you have?
Python Statistics Pandas
<s>[INST] With this area of interest Btech Computer Science and skillset Python Statistics Pandas what can be my suitable career? [/INST]  With a B. everybody Technology in Computer Science and a skillset in Python and Statistics, particularly Pandas, you have a versatile combination of skills that can lead to a variety of career paths. Here are some suitable career options to consider:

1. Data Scientist: As a data scientist, you will work with large datasets to extract insights and build predictive models using Python and Pandas. You will also work on data visualization, data preprocessing, and data analysis.
2. Data Analyst: As a data analyst, you will work with data to identify trends, patterns, and insights using Python and Pandas. You will al

END